In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sdv.tabular import CopulaGAN
from sdv.evaluation import evaluate
from sdv.constraints import UniqueCombinations, GreaterThan


import os, glob

In [8]:
data = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/Survival_df.csv'),
                    index_col=0)
#data.drop(columns=['NLE_ratio_119_17'],inplace=True)

In [9]:
data.NDE_cycle = data.NDE_cycle.astype('str')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 41C to 49C
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Observed              21 non-null     bool   
 1   F_Time                21 non-null     float64
 2   NDE_cycle             21 non-null     object 
 3   diff_two_peaks_ratio  21 non-null     float64
 4   amp_ratio             21 non-null     float64
 5   pos_ratio             21 non-null     float64
 6   energy_ratio          21 non-null     float64
 7   NLE_ratio_51_17       21 non-null     float64
 8   NLE_ratio_85_17       21 non-null     float64
 9   NLE_ratio_51_119      21 non-null     float64
 10  NLO_avg               21 non-null     float64
 11  Avg_RP                21 non-null     float64
 12  ACEE_30_Avg_58        21 non-null     float64
 13  ACEE_500_Avg_58       21 non-null     float64
 14  ACPD_Avg_58           21 non-null     float64
dtypes: bool(1), float64(13), ob

# CopulaGAN

The sdv.tabular.CopulaGAN model is a variation of the CTGAN Model which takes advantage of the CDF based transformation that the GaussianCopulas apply to make the underlying CTGAN model task of learning the data easier.

# Model the data

## tuning distribution and fitting model

In [11]:
model = CopulaGAN(
    epochs=5000,
    field_distributions={
        'diff_two_peaks_ratio':'gaussian_kde',
        'energy_ratio':'gaussian_kde',
        'ACEE_500_Avg_58':'gaussian_kde',
        'amp_ratio':'gaussian_kde',
        'NLE_ratio_51_17':'gaussian_kde',
        'NLE_ratio_85_17':'gaussian_kde',
        'NLE_ratio_119_17':'gaussian_kde',
    }
)

In [12]:
model.fit(data)

/home/marie-anne/anaconda3/envs/EPRI/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:621: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/marie-anne/anaconda3/envs/EPRI/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/marie-anne/anaconda3/envs/EPRI/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:4965: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/home/marie-anne/anaconda3/envs/EPRI/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2429: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/home/marie-anne/anaconda3/envs/EPRI/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keywo

In [13]:
model.get_distributions()

{'Observed': 'copulas.univariate.gaussian.GaussianUnivariate',
 'F_Time': 'copulas.univariate.beta.BetaUnivariate',
 'diff_two_peaks_ratio': 'copulas.univariate.gaussian_kde.GaussianKDE',
 'amp_ratio': 'copulas.univariate.gaussian_kde.GaussianKDE',
 'pos_ratio': 'copulas.univariate.uniform.UniformUnivariate',
 'energy_ratio': 'copulas.univariate.gaussian_kde.GaussianKDE',
 'NLE_ratio_51_17': 'copulas.univariate.gaussian_kde.GaussianKDE',
 'NLE_ratio_85_17': 'copulas.univariate.gaussian_kde.GaussianKDE',
 'NLE_ratio_51_119': 'copulas.univariate.student_t.StudentTUnivariate',
 'NLO_avg': 'copulas.univariate.gamma.GammaUnivariate',
 'Avg_RP': 'copulas.univariate.gamma.GammaUnivariate',
 'ACEE_30_Avg_58': 'copulas.univariate.log_laplace.LogLaplace',
 'ACEE_500_Avg_58': 'copulas.univariate.gaussian_kde.GaussianKDE',
 'ACPD_Avg_58': 'copulas.univariate.gamma.GammaUnivariate'}

# Generate synthetic data

In [14]:
samples = model.sample(1000)

In [16]:
samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Observed              1000 non-null   bool   
 1   F_Time                1000 non-null   float64
 2   NDE_cycle             1000 non-null   object 
 3   diff_two_peaks_ratio  1000 non-null   float64
 4   amp_ratio             1000 non-null   float64
 5   pos_ratio             1000 non-null   float64
 6   energy_ratio          1000 non-null   float64
 7   NLE_ratio_51_17       1000 non-null   float64
 8   NLE_ratio_85_17       1000 non-null   float64
 9   NLE_ratio_51_119      1000 non-null   float64
 10  NLO_avg               1000 non-null   float64
 11  Avg_RP                1000 non-null   float64
 12  ACEE_30_Avg_58        1000 non-null   float64
 13  ACEE_500_Avg_58       1000 non-null   float64
 14  ACPD_Avg_58           1000 non-null   float64
dtypes: bool(1), float64(13

### Evaluate

The output of this function call will be a number between 0 and 1 that will indicate us how similar the two tables are, being 0 the worst and 1 the best possible score.

The evaluate function applies a collection of pre-configured metric functions and returns the average of the scores that the data obtained on each one of them. To explore the metrics in more detail, you can pass and additional argument aggregate=False.


- cstest: This metric compares the distributions of all the categorical columns of the table by using a Chi-squared test and returns the average of the p-values obtained across all the columns. If the tables that you are evaluating do not contain any categorical columns the result will be nan.

- kstest: This metric compares the distributions of all the numerical columns of the table with a two-sample Kolmogorov–Smirnov test using the empirical CDF and returns the average of the p-values obtained across all the columns. If the tables that you are evaluating do not contain any numerical columns the result will be nan.

- logistic_detection: This metric tries to use a Logistic Regression classifier to detect whether each row is real or synthetic and then evaluates its performance using an Area under the ROC curve metric. The returned score is 1 minus the ROC AUC score obtained by the classifier.

- svc_detection: This metric tries to use an Support Vector Classifier to detect whether each row is real or synthetic and then evaluates its performance using an Area under the ROC curve metric. The returned score is 1 minus the ROC AUC score obtained by the classifier.


In [17]:
evaluate(samples, data)

0.6807934189707292

In [18]:
evaluate(samples, data, aggregate = False)

{'cstest': 0.9790888009727357,
 'kstest': 0.3557752650436851,
 'logistic_detection': 0.6640078915528017,
 'svc_detection': 0.6147618961990219}

In [21]:
samples.NDE_cycle = samples.NDE_cycle.astype('int')
samples.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/CopulaGAN_simulated_data_survival_2.csv'), index=False)

# 4. Save and load the synthesizer

To save a trained ctgan synthesizer, use

`model.save('my_model.pkl')`

To restore a saved synthesizer, use

`loaded = CopulaGAN.load('my_model.pkl')`

`new_data = loaded.sample(200)`

In [23]:
model.save('CopulaGAN_survival_2.pkl')